# PREPROCESSING DE LOS DATOS

En este notebook vamos a ver cómo preprocesar los datos usando scikit. Cuando hacemos preprocesado de los datos, se realizan diferentes transformaciones sobre éstos, bien para eliminar o reemplazar información no útil, o bien para que los algoritmos de clasifiación funcionen correctamente. Por ejemplo, algoritmos como knn, logistic regression (con penalty) y support vector machine necesitan que los datos tengan la misma escala

Scikit posee el módulo `preprocessing`, el cual contiene numerosas herramientas para llevar a cabo operaciones de transformación de datos, y el módulo `impute` (antes en `preprocessing`), para rellenar los huecos en nuestros datos 

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
dataframe = pd.read_csv("https://raw.githubusercontent.com/jrasero/curso-scikit-ehu-2019/master/datasets/pima-indians-diabetes.csv")

## Trabajando con NANs: Módulo `impute` (antes en módulo `preprocessing`)

In [ ]:
# Los datos contienen NaNs
dataframe.info()

In [ ]:
dataframe.dtypes

Una opción podría ser eliminar directamente las filas que contengan NaNs. 

In [ ]:
dataframe_clean = dataframe.dropna()

Lo malo de este approach es que podemos perder observaciones que pueden ser valiosas 

In [ ]:
#se pierden muchos datos
print(dataframe_clean.shape)

Podríamos intentar rellenar estos huecos reemplazando por algún valor representativo de la columna donde se encuentra el hueco

In [ ]:
from sklearn.impute import SimpleImputer
imp = SimpleImputer(strategy='mean')
mat_clean=imp.fit_transform(dataframe.values)
print(mat_clean.shape)

In [ ]:
#Usando Pandas
dataframe.fillna(dataframe.mean()).shape

## Reescaleando los datos: Módulo `preprocessing`

Al ajustar muchos algoritmos, los datos tienen que encontrarse en una misma escala para poder encontrar una solución óptima. El módulo preprocessing de scikit nos permite ocuparnos de esto de manera muy sencilla

In [ ]:
X = mat_clean[:,0:8]
y = mat_clean[:,8]

In [ ]:
# Los datos están en una escala diversa
print(X.mean(axis=0))
print(X.min(axis=0))
print(X.max(axis=0))

In [ ]:
# Reescalar data (Entre 0 and 1)
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0, 1))
rescaledX = scaler.fit_transform(X)
np.set_printoptions(precision=3)
print(rescaledX[0:5,:])

In [ ]:
# Estandarizar data (0 mean, 1 stdev)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
rescaledX = scaler.fit_transform(X)
print(rescaledX[0:5,:])

In [ ]:
# Normalizar data
from sklearn.preprocessing import Normalizer
scaler = Normalizer()
normalizedX = scaler.fit_transform(X)
np.set_printoptions(precision=3)
print(normalizedX[0:5,:])

In [ ]:
# binarizar los datos
from sklearn.preprocessing import Binarizer
binarizer = Binarizer(threshold=0.0)
binaryX = binarizer.fit_transform(X)
np.set_printoptions(precision=3)
print(binaryX[0:5,:])

### EFECTOS DEL PREPROCESSING

Manejo de diferentes tipos de datos

    Hay tres tipos de tipo de datos:
        Numericos, e.g. income, age
        Categóricos o nominales, e.g. gender, nationality
        Ordinales, e.g. low/medium/high

    En scikit solo features numéricas

    Debemos convertir las variables categóricas y ordinales en numéricas
        Crear dummy features
        Transformar una feature categórica en un grupo de dummy features, cada una representando una única categoría 
        De esta manera, en un conjunto de dummy features, 1 indica que una observación pertenece a esa categoría

Ejemplo extraído de :
https://www.analyticsvidhya.com/blog/2016/07/practical-guide-data-preprocessing-python-scikit-learn/

In [ ]:
X_train_df = pd.read_csv('https://raw.githubusercontent.com/jrasero/curso-scikit-ehu-2019/master/datasets/loan_train.csv')
y_train_df = pd.read_csv('https://raw.githubusercontent.com/jrasero/curso-scikit-ehu-2019/master/datasets/loan_target_train.csv')
X_test_df = pd.read_csv('https://raw.githubusercontent.com/jrasero/curso-scikit-ehu-2019/master/datasets/loan_test.csv')
y_test_df = pd.read_csv('https://raw.githubusercontent.com/jrasero/curso-scikit-ehu-2019/master/datasets/loan_target_test.csv')

In [ ]:
print (X_train_df.head())

In [ ]:
y_train_df.head()

In [ ]:
# Convertimos primero los targets a números
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(y_train_df.values)
y_test = le.transform(y_test_df.values)

In [ ]:
# Miremos sólo por ahora las variables numéricas
numerical_cols = ['ApplicantIncome', 'CoapplicantIncome','LoanAmount', 
                   'Loan_Amount_Term', 'Credit_History']

In [ ]:
# KNN es un clasificador sensible a las diferencias entre escalas
# (al usar distancias)
from sklearn.neighbors import KNeighborsClassifier
# Definición del clasificador
knn_clf=KNeighborsClassifier(n_neighbors=5)

# Ajustamos el clasificador al train
knn_clf.fit(X_train_df.loc[:, numerical_cols].values, y_train)

# Predecimos sobre el test
print("accuracy = ", 
      knn_clf.score(X_test_df.loc[:, numerical_cols].values,y_test))

In [ ]:
# Veamos lo mismo poniendo todos a la misma escala
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
# Escalamos tanto el train con el test set
X_train_minmax=scaler.fit_transform(X_train_df.loc[:, numerical_cols].values)
X_test_minmax=scaler.transform(X_test_df.loc[:, numerical_cols].values) # Ojo el transform aqui!

In [ ]:
# Ajustamos el clasificador al train
knn_clf.fit(X_train_minmax,y_train)

# Predecimos sobre el test
print(knn_clf.score(X_test_minmax,y_test))

In [ ]:
# Logistic Regression también es sensible a las diferentes escalas de las 
# features
from sklearn.linear_model import LogisticRegression

In [ ]:
# Definición del clasificador
log_clf = LogisticRegression(penalty='l2', C=0.01, random_state=0)

# Ajustamos el clasificador al train
log_clf.fit(X_train_df.loc[:,numerical_vars].values, y_train)

# Predecimos sobre el test
print(" Logistic regression antes de preprocessing: ", 
      log_clf.score(X_test_df.loc[:,numerical_cols].values, y_test))
      
log_clf.fit(X_train_minmax, y_train)
print(" Logistic regression después de preprocessing ", 
      log_clf.score(X_test_minmax,y_test))

In [ ]:
# A veces depende de cómo normalicemos
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()

X_train_scale=ss.fit_transform(X_train_df.loc[:,numerical_cols].values)

X_test_scale=ss.transform(X_test_df.loc[:,numerical_cols].values)

log_clf.fit(X_train_scale,y_train)
print("Logistic Regression estandarizando: ", 
      log_clf.score(X_test_scale, y_test))

Algunos valores son categóricos. Para poder trabajar con ellos, hace falta primero codificarlos como números y después crear un conjunto de dummy features, de tal forma que cada categoría dentro de una misma variable sea una columns

In [ ]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()

In [ ]:
# Por ejemplo la variable Property_Area
property_col = X_train_df.loc[:, "Property_Area"].values
print(property_col[20:30])

property_col_le = le.fit_transform(property_col)
print( property_col_le[20:30])

Pero claro, así muchos clasificadores supondría que entre la categoría Urban
y Rural hay el doble de distancia que entre Urban y Semiurban. Esto no es del todo correcto. Podemos pensar en ciudades también. Por ejemplo, Bilbo, Gazteiz y Donosti. Si las codificamos como 0, 1, 2, tiene sentido que entre Bilbo y Donosti hay el doble de valor que Bilbo-Gazteiz?

In [ ]:
X_test_df.dtypes

Hay que crear un conjunto de dummy features, de tal forma que cada categoría dentro de una misma variable sea una columns. Esto se puede hacer con la clase `OneHotEncoder` en el módulo preprocessing 

In [ ]:
# para la columna anterior, por ejemplo
from sklearn.preprocessing import OneHotEncoder
enc=OneHotEncoder(sparse=False)
enc.fit_transform(property_col_le.reshape(-1,1))

In [ ]:
cat_cols=['Gender', 'Married', 'Dependents', 'Education','Self_Employed',
          'Credit_History', 'Property_Area']

In [ ]:
X_train_df.loc[:, cat_cols].head()

In [ ]:
X_train_df.drop(columns=cat_cols)

In [ ]:
enc.fit_transform(X_train_df.loc[:, cat_cols].values)

In [ ]:
X_train_ohe = np.concatenate([X_train_df.loc[:,numerical_cols].values,
          enc.fit_transform(X_train_df.loc[:, cat_cols].values)],
         axis=1)

X_test_ohe = np.concatenate([X_test_df.loc[:,numerical_cols].values,
          enc.transform(X_test_df.loc[:, cat_cols].values)],
         axis=1)

In [ ]:
X_train_scale=ss.fit_transform(X_train_ohe)
X_test_scale=ss.transform(X_test_ohe)

log_clf.fit(X_train_scale, y_train)

log_clf.score(X_test_scale, y_test)

Para estos casos, yo recomiendo mejor usar pandas, que te permite hacer el label y one hot enconding fácilmente usando el método `get_dummies`

In [ ]:
X_train_df = pd.read_csv('https://raw.githubusercontent.com/jrasero/curso-scikit-ehu-2019/master/datasets/loan_train.csv')
y_train_df = pd.read_csv('https://raw.githubusercontent.com/jrasero/curso-scikit-ehu-2019/master/datasets/loan_target_train.csv')
X_test_df = pd.read_csv('https://raw.githubusercontent.com/jrasero/curso-scikit-ehu-2019/master/datasets/loan_test.csv')
y_test_df = pd.read_csv('https://raw.githubusercontent.com/jrasero/curso-scikit-ehu-2019/master/datasets/loan_target_test.csv')

In [ ]:
pd.get_dummies(X_train_df['Married'])

In [ ]:
pd.get_dummies(X_train_df, columns=cat_cols, drop_first=False).head()

In [ ]:
X_train_dummies = pd.concat([X_train_df.loc[:, numerical_cols],
                             pd.get_dummies(X_train_df.loc[:, cat_cols])], 
                            axis=1)

Hemos visto como la performance del clasificador puede cambiar según cómo manejemos los datos. No hay forma única y a veces es complicado saber qué procesamiento se debe adoptar. Algunos casos, como Decision Tree y Random Forest, apenas requieren mucho preprocessing. Otros, como support vector machine, logistic regression y knn requieren tratar los datos categóricos y poner todos los datos en la misma escala. Para estos casos, decidir sobre si estandarizar o sólo escalar los datos entre 0 y 1 depende de la naturaleza de los datos en si. Lo mejor, al principio es adoptar las diferentes posibilidades, comparar la performance en cada  y quedarte con el mejor de los casos

Referencias:

- http://scikit-learn.org/stable/modules/preprocessing.html